# Asphalt mix prediction after plastic addition

In [283]:
#%% IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import seaborn as sns
from scipy.stats import normaltest
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from IPython.display import display, Markdown, Latex
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge


Data Import 

In [ ]:
#%%DATA READING AND INITIAL PREPROCESSING
numericColumns = ['Aggregate absorption [%]',
                  'Apparent specific gravity',
                    0.075,
                    0.3,
                    0.6,
                    2.36,
                    4.75,
                    9.5,
                    12.5,
                    19,
                    'Plastic particle size (mm)',
                    'Mixing speed (RPM)',
                    'Mixing Temperature',
                    'Mixing Time (hours)',
                    'Plastic Addition by bitumen weight (%)',
                    ]
categoricalColumns = ['Modified asphaly Mix?',
                      'Agreggate Type',
                    'Aggregate absorption [%]',
                    'Filler used',
                    'Consolidated bitumen penetration grade',
                    'New Plastic Type',
                    'Plastic pretreatment',
                    'Plastic shape',
                    'Plastic Size',
                    'Mixing Process',
                    'Plastic melted previous to addition?',
                    'Aggregates replacement ?',
                    'Bitumen replacement?',
                    'Filler replacement',
                    'Property',
                    'Units']
#It returns the dataframe of interes based on the property - 'AirVoids', 'MS', 'MF', 'ITS', 'TSR'
def returnDf (propertyOfInterest):
    df = pd.read_excel('fileML.xlsx', sheet_name = propertyOfInterest, engine='openpyxl')
    df = df.set_index(propertyOfInterest + ' ID')
    df.loc[:,:'Units'] = df.loc[:,:'Units'].applymap(str)
    df.loc[:,:'Units'] = df.loc[:,:'Units'] .applymap(str.strip)
    df.replace('NS', np.nan, inplace = True)
    df[numericColumns] = df[numericColumns].replace('N/a', 0).astype(float)
    return df

In [ ]:
dfAirVoids = returnDf('AirVoids')
dfMS = returnDf('AirVoids')
dfMF = returnDf('MF')
dfITS = returnDf('ITS')
dfTSR = returnDf('TSR')

-------------------------------
#  1. Air Voids

## 1.1 Data Exploration

In [ ]:
#Eliminate Outliers
Q1 = dfAirVoids["Air voids of the sample (%)"].quantile(0.25)
Q3 = dfAirVoids["Air voids of the sample (%)"].quantile(0.75)
IQR = Q3 - Q1
dfAirVoids = dfAirVoids[(dfAirVoids["Air voids of the sample (%)"] < (Q3 + 1.5 * IQR)) & (dfAirVoids["Air voids of the sample (%)"] > (Q1 - 1.5 * IQR))]
dfAirVoids.shape

###  1.1.1 Total Sample

In [ ]:
dfAirVoids.info()

In [ ]:
columnsOfInteres = numericColumns[0:2]+numericColumns[10:]+['Air voids of the sample (%)']

In [ ]:
dfAirVoids.describe(include = "all")

In total, there is $\color{red}{\text{288}}$ observations. Let's see if there is any correlation with the continual and categorical variables.

In [ ]:
scatter_matrix(dfAirVoids[['Aggregate absorption [%]', 'Apparent specific gravity', 'Air voids of the sample (%)']], figsize=(10, 10))
plt.show()

Among the categorical variables, we will initially focus on the general ones that might have an effect on air voids: $\color{red}{\text{Aggregate type}}$, $\color{red}{\text{Filler used}}$ and $\color{red}{\text{Bitumen Grade}}$

In [ ]:
#CORRELATION:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(dfAirVoids.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,10))
sns.boxplot(y="Air voids of the sample (%)", x= "Agreggate Type", data=dfAirVoids,  orient='v' , ax=ax1)
sns.boxplot(y="Air voids of the sample (%)", x= "Filler used", data=dfAirVoids,  orient='v' , ax=ax2)
sns.boxplot(y="Air voids of the sample (%)", x= "Consolidated bitumen penetration grade", data=dfAirVoids,  orient='v' , ax=ax3)
sns.boxplot(y="Air voids of the sample (%)", x= "Modified asphaly Mix?", data=dfAirVoids,  orient='v' , ax=ax4)

Within the  $\color{red}{\text{Aggregate type}}$, although it exists differences -especially with the Ophitic- this data migh not be robust enough due to the sample size of this type of aggregate. It happens the same with the  $\color{red}{\text{Bitumen type}}$, where only  $\color{purple}{\text{5}}$ observations used the 40/50 bitumen. The  $\color{red}{\text{addition of plastic}}$ seems to have not strong influence in the air voids.

###  1.1.2 Modified mixtures

In [ ]:
dfAirVoidsModified = dfAirVoids[dfAirVoids['Modified asphaly Mix?']=='Yes']
dfAirVoidsModified.describe(include = "all")

In [ ]:
scatter_matrix(dfAirVoidsModified[columnsOfInteres], figsize=(25, 20))
plt.show()

In [ ]:
#CORRELATION:
plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(dfAirVoids.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15,10))
sns.boxplot(y="Air voids of the sample (%)", x= "Agreggate Type", data=dfAirVoidsModified,  orient='v' , ax=ax1)
sns.boxplot(y="Air voids of the sample (%)", x= "Plastic shape", data=dfAirVoidsModified,  orient='v' , ax=ax2)
sns.boxplot(y="Air voids of the sample (%)", x= "New Plastic Type", data=dfAirVoidsModified,  orient='v' , ax=ax3)
sns.boxplot(y="Air voids of the sample (%)", x= "Mixing Process", data=dfAirVoidsModified,  orient='v' , ax=ax4)

Not much difference among dry and wet

###  1.1.3 Wet vs. Dry Mixing

In [ ]:
dfAirVoidsWetvsDry = dfAirVoidsModified [['Mixing Process', 'Air voids of the sample (%)']]
dfAirVoidsWetvsDry.groupby(['Mixing Process'], as_index=False).describe()

In [ ]:
sns.pairplot(dfAirVoidsModified[columnsOfInteres+['Mixing Process']], hue="Mixing Process", height=2.5)

##  **Some important points:**

*   There are missing values mainly in $\color{red}{\text{Apparent specific gravity}}$, $\color{red}{\text{Aggregate type}}$ and $\color{red}{\text{filler used}}$.
*   Total sample: # observations = 288, Mean = 4, Standard deviation = 0.98 
*   Sample with asphalt modified: # observations = 228, Mean = 3.94, Standard deviation = 1.017
*   It seems that there will be a strong correlation with $\color{red}{\text{Aggregates gradation}}$.
*   Dry vs. Wet:
    *   Not strong difference between dry and wet.
    *   Mean in wet = 3.64, Mean in dry = 4.16.


## 1.2 Data Pre-processing

### 1.2.1 Categorical variables and Missing Values

In [49]:
dfAirVoids.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 305
Data columns (total 35 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Article ID                              288 non-null    object 
 1   Global ID                               288 non-null    object 
 2   Modified asphaly Mix?                   288 non-null    object 
 3   Agreggate Type                          197 non-null    object 
 4   Aggregate absorption [%]                231 non-null    float64
 5   Apparent specific gravity               77 non-null     float64
 6   0.075                                   254 non-null    float64
 7   0.3                                     254 non-null    float64
 8   0.6                                     251 non-null    float64
 9   2.36                                    258 non-null    float64
 10  4.75                                    258 non-null    float6

*   Process for dealing with missing categorical values:

    1.  I will eliminate the rows with Nan values in the column $\color{red}{\text{New Plastic Type}}$. In total, there is 24 missing values, which represent 8.33% of the sample
    2.  Replace the 'N/a' value when there is no modifiation of the asphalt mix.

*   Process for dealing with missing numeric values:

    *   The imputer used was the multivariable imputer, and the estimator was the ExtraTreeRegressor ([link][1]).
    By using this estimator, I did not get any negative values among the missing features, and it presents a low MSE value.
    
[1]:https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py  

In [51]:
#CATEGORICAL VARIABLES
dfAirVoidsCleaned = dfAirVoids.drop(['Article ID', 'Global ID','Agreggate Type', 'Apparent specific gravity', 'Filler used', 'Bitumen Type Penetration Grade', 'Property', 'Units', 'Plastic Size' ], axis = 1)
dfAirVoidsCleaned = dfAirVoidsCleaned.replace('N/a', 0)
dfAirVoidsCleaned = dfAirVoidsCleaned.dropna(subset=['New Plastic Type'])
dfAirVoidsCleaned = pd.get_dummies(dfAirVoidsCleaned, drop_first=True)
dfAirVoidsCleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 264 entries, 1 to 305
Data columns (total 40 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Aggregate absorption [%]                       207 non-null    float64
 1   0.075                                          254 non-null    float64
 2   0.3                                            254 non-null    float64
 3   0.6                                            251 non-null    float64
 4   2.36                                           258 non-null    float64
 5   4.75                                           258 non-null    float64
 6   9.5                                            251 non-null    float64
 7   12.5                                           243 non-null    float64
 8   19                                             258 non-null    float64
 9   Plastic particle size (mm)                     238 non

In [52]:
#IMPUTATION OF MISSING VALUES
imputer = IterativeImputer (estimator = ExtraTreesRegressor(n_estimators=10, random_state=0), max_iter=50)
n = imputer.fit_transform(dfAirVoidsCleaned)
dfCleanedImputed = pd.DataFrame(n, columns = list(dfAirVoidsCleaned.columns))
print ('There is '+str(sum(n < 0 for n in dfCleanedImputed.values.flatten()))+' negative values in the new Dataframe')

C:\Users\cavs1\Anaconda3\envs\spyder\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


### 1.2.2 Features Scaling

In [63]:
 scaler = MinMaxScaler()
 dfCleanedImputedScaled = pd.DataFrame(scaler.fit_transform(dfCleanedImputed), columns = list(dfCleanedImputed.columns))

## 1.3 Model Training

In [75]:
X = dfCleanedImputedScaled.loc[:, dfCleanedImputedScaled.columns != 'Air voids of the sample (%)']
y = dfCleanedImputedScaled.iloc[:,15]

In [280]:
#Method for displaying the MSE and best parameters
def displayScore (grid):
    display(Markdown('Lowest MSE:'+ str(-grid.best_score_)))
    display(Markdown('Best parameters: ' +str(grid.best_params_)))

### Linear Model

In [281]:
param_grid = {'fit_intercept': [True, False],
            'positive': [True, False]}
grid = GridSearchCV(LinearRegression(), param_grid, cv=10, scoring='neg_mean_squared_error')
grid.fit(X, y)
displayScore(grid)

Lowest MSE:0.03873642008197666

Best parameters: {'fit_intercept': False, 'positive': True}

### Ridge linear regression model

In [297]:
param_grid = {'alpha': [80,81,82,83,84],
'fit_intercept': [True, False],
'solver': ['svd', 'cholesky','lsqr', 'sparse_cg']}
grid = GridSearchCV(Ridge(), param_grid, cv=10, scoring='neg_mean_squared_error')
grid.fit(X, y)
displayScore(grid)

Lowest MSE:0.03242478861927641

Best parameters: {'alpha': 80, 'fit_intercept': True, 'solver': 'svd'}

### Polynomial Model

In [304]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))


In [ ]:
param_grid = {'polynomialfeatures__degree': [3,4],
'linearregression__fit_intercept': [True, False],
'linearregression__positive':[True, False]}
grid = GridSearchCV(PolynomialRegression(), param_grid, cv=10, scoring='neg_mean_squared_error')
grid.fit(X, y)
displayScore(grid)

### Ridge polynomial regression

In [298]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), Ridge(**kwargs))

In [302]:
param_grid = {'polynomialfeatures__degree': [2, 3],
'ridge__alpha':[0.001,0.1,1,10],
'ridge__fit_intercept': [True, False]}
grid = GridSearchCV(PolynomialRegression(), param_grid, cv=10, scoring='neg_mean_squared_error')
grid.fit(X, y)
displayScore(grid)

Lowest MSE:0.04027501977634906

Best parameters: {'polynomialfeatures__degree': 2, 'ridge__alpha': 10, 'ridge__fit_intercept': False}

---------------------------------
###  2.2 Marshall Stability

In [ ]:
dfMS.describe(include = "all")

In [ ]:
dfMF.describe(include = "all")


In [ ]:
dfITS.describe(include = "all")


In [ ]:
dfTSR.describe(include = "all")